# ip data and binary data

In [29]:
from sklearn.utils import shuffle
import pandas as pd

def toint(row):
    row['1st Octet']=int(row['1st Octet'])
    row['2nd Octet']=int(row['2nd Octet'])
    row['3rd Octet']=int(row['3rd Octet'])
    row['4th Octet']=int(row['4th Octet'])
    return row

def ipcleanup(row):
    iprow=[]
    for address in row['A']:
        try:
            tempstring=address.split('IN A')
            iprow.append(tempstring[1].strip())
        except IndexError:  
            pass
    return iprow

def binarizer(cell):
    if cell[0]=="No Answer" or cell[0]=="Timeout":
        return 0
    else:
        return 1

goodcomplete=pd.read_parquet('../parsed data/gooddnsdata')
goodcomplete=goodcomplete.sample(66000)
badcomplete=pd.read_parquet('../parsed data/baddnsdata')

goodcomplete["indicator"]=goodcomplete["indicator"].apply(lambda x:x[0])
goodcomplete.insert(0,'Malicious',0)
badcomplete.insert(0,'Malicious',1)

badcomplete["indicator"]=badcomplete["indicator"].apply(lambda x: x[0])
badcomplete=badcomplete.drop_duplicates(subset="indicator")
##############################################################################
goodcomplete['A']=goodcomplete.apply(ipcleanup,axis="columns")
badcomplete['A']=badcomplete.apply(ipcleanup,axis="columns")

complete = goodcomplete.append(badcomplete,ignore_index=True)
#cut out rows with empty ip
complete=complete[complete['A'].map(lambda x: len(x)>0)]

complete['A']=complete['A'].apply(lambda x:x[0])
expand=complete['A'].str.split(".",-1,expand=True)
complete["1st Octet"]=expand[0]
complete["2nd Octet"]=expand[1]
complete["3rd Octet"]=expand[2]
complete["4th Octet"]=expand[3]

#reorder
complete=complete[['indicator','Malicious', '1st Octet',"2nd Octet","3rd Octet","4th Octet",'A', 'AAAA', 'CAA', 'CNAME', 'DNAME', 'LOC', 'MX','RP', 'SIG', 'SSHFP' ]]
complete[['A', 'AAAA', 'CAA', 'CNAME', 'DNAME', 'LOC', 'MX','RP', 'SIG', 'SSHFP' ]]=complete[['A', 'AAAA', 'CAA', 'CNAME', 'DNAME', 'LOC', 'MX','RP', 'SIG', 'SSHFP' ]].applymap(binarizer)

In [33]:
from sklearn.preprocessing import OneHotEncoder
complete[['1st Octet',"2nd Octet","3rd Octet","4th Octet"]]=complete[['1st Octet',"2nd Octet","3rd Octet","4th Octet"]].astype(int)

complete=complete.sample(frac=1)

train=complete[0:80000]
test=complete[80000:]

cats=[list(range(0,256)),list(range(0,256)),list(range(0,256)),list(range(0,256)),[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]

encoder=OneHotEncoder(categories=cats)
encodedtrain=encoder.fit_transform(train[["1st Octet","2nd Octet","3rd Octet","4th Octet",'AAAA', 'CAA', 'CNAME', 'DNAME', 'LOC', 'MX', 'RP', 'SIG',
      'SSHFP']])
encodedtest=encoder.fit_transform(test[["1st Octet","2nd Octet","3rd Octet","4th Octet",'AAAA', 'CAA', 'CNAME', 'DNAME', 'LOC', 'MX', 'RP', 'SIG',
      'SSHFP']])


# Logistic Regression

In [34]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
glm=LogisticRegression()

model=glm.fit(encodedtrain,train['Malicious'])
ypred=model.predict(encodedtest)
print(classification_report(test['Malicious'],ypred))


/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.75      0.81      0.78     21603
           1       0.78      0.71      0.74     20597

   micro avg       0.76      0.76      0.76     42200
   macro avg       0.76      0.76      0.76     42200
weighted avg       0.76      0.76      0.76     42200



# MLP 

In [35]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

clf=MLPClassifier()

model=clf.fit(encodedtrain,train['Malicious'])
ypred=model.predict(encodedtest)
yprob=model.predict_proba(encodedtest)
print(classification_report(test['Malicious'],ypred))


              precision    recall  f1-score   support

           0       0.80      0.83      0.81     21603
           1       0.81      0.78      0.79     20597

   micro avg       0.80      0.80      0.80     42200
   macro avg       0.80      0.80      0.80     42200
weighted avg       0.80      0.80      0.80     42200



In [ ]:
import matplotlib.pyplot as plt
fpr,tpr,thresholds=roc_curve(test['Benign'],yprob[:,0],pos_label=0)

plt.plot(fpr,tpr)

plt.xlabel("tpr")
plt.ylabel("fpr")
plt.show()



In [20]:
len(train[train["Malicious"]==1])

41585

# RANDOM FOREST

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
rfc=RandomForestClassifier()
model=rfc.fit(encodedtrain,train['Malicious'])
ypred=model.predict(encodedtest)
print(classification_report(test['Malicious'],ypred))


/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.80      0.88      0.84     21603
           1       0.86      0.77      0.81     20597

   micro avg       0.82      0.82      0.82     42200
   macro avg       0.83      0.82      0.82     42200
weighted avg       0.83      0.82      0.82     42200



# QDA

In [ ]:
from sklearn.svm import SVC
supportvec=SVC()
model=supportvec.fit(encodedtrain,train['Benign'])
ypred=model.predict(encodedtest)
print(classification_report(test['Benign'],ypred))


# NEAREST NEIGHBOR

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier()
model=knn.fit(encodedtrain,train['Benign'])
ypred=model.predict(encodedtest)
print(classification_report(test['Benign'],ypred))
